In [1]:
from PIL import Image
import numpy as np
import os
from random import shuffle
import matplotlib.pyplot as plt

In [2]:
train_ktp = "selfie_ktp/train_selfie"
test_ktp = "selfie_ktp/test_selfie"

In [3]:
# def get_size_statistics():
#     heights = []
#     widths = []
#     img_count = 0
#     for img in os.listdir(train_ktp):
#         path = os.path.join(train_ktp, img)
#         if "DS_Store" not in path:
#             data = np.array(Image.open(path))
#             heights.append(data.shape[0])
#             widths.append(data.shape[1])
#             img_count += 1
#     avg_height = sum(heights) / len(heights)
#     avg_width = sum(widths) / len(widths)
#     print("Average Height: " + str(avg_height))
#     print("Max Height: " + str(max(heights)))
#     print("Min Height: " + str(min(heights)))
#     print('\n')
#     print("Average Width: " + str(avg_width))
#     print("Max Width: " + str(max(widths)))
#     print("Min Width: " + str(min(widths)))

# get_size_statistics()

In [4]:
def label_img(name):
    word_label = name.split('-')[0]
    if word_label == 'fix_selfie': return np.array([1, 0, 0])
    elif word_label == 'not_selfie' : return np.array([0, 1, 0])
    elif word_label == 'blured_selfie' : return np.array([0, 0, 1])

In [5]:
IMG_SIZE = 256

def load_training_data():
    train_data = []
    for img in os.listdir(train_ktp):
        label = label_img(img)
        path = os.path.join(train_ktp, img)
        if "DS_Store" not in path:
            img = Image.open(path)
            img = img.convert('L')
            img = img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
            train_data.append([np.array(img), label])
            
    shuffle(train_data)
    return train_data

In [6]:
train_data = load_training_data()
plt.imshow(train_data[43][0], cmap = 'gist_gray')

In [7]:
trainImages = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
trainLabels = np.array([i[1] for i in train_data])

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers. normalization import BatchNormalization
import numpy as np

Using TensorFlow backend.


In [9]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(3, activation = 'softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [13]:
model.fit(trainImages, trainLabels, batch_size = 50, epochs = 50, verbose = 1)

Epoch 1/50
4017/4017 [==============================] - 325s 81ms/step - loss: 0.4074 - acc: 0.8144
Epoch 2/50
4017/4017 [==============================] - 322s 80ms/step - loss: 0.3090 - acc: 0.8658
Epoch 3/50
4017/4017 [==============================] - 322s 80ms/step - loss: 0.2297 - acc: 0.9043
Epoch 4/50
4017/4017 [==============================] - 322s 80ms/step - loss: 0.1883 - acc: 0.9266
Epoch 5/50
4017/4017 [==============================] - 325s 81ms/step - loss: 0.1573 - acc: 0.9381
Epoch 6/50
4017/4017 [==============================] - 326s 81ms/step - loss: 0.1231 - acc: 0.9523
Epoch 7/50
4017/4017 [==============================] - 323s 81ms/step - loss: 0.0943 - acc: 0.9670
Epoch 8/50
4017/4017 [==============================] - 325s 81ms/step - loss: 0.0750 - acc: 0.9725
Epoch 9/50
4017/4017 [==============================] - 320s 80ms/step - loss: 0.0628 - acc: 0.9788
Epoch 10/50
4017/4017 [==============================] - 325s 81ms/step - loss: 0.0519 - acc: 0.9818

In [14]:
def load_test_data():
    test_data = []
    for img in os.listdir(test_ktp):
        label = label_img(img)
        path = os.path.join(test_ktp, img)
        if "DS_Store" not in path:
            img = Image.open(path)
            img = img.convert('L')
            img = img.resize((IMG_SIZE, IMG_SIZE), Image.ANTIALIAS)
            test_data.append([np.array(img), label])
    shuffle(test_data)
    return test_data


test_data = load_test_data()    
plt.imshow(test_data[10][0], cmap = 'gist_gray')

In [15]:
testImages = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
testLabels = np.array([i[1] for i in test_data])

loss, acc = model.evaluate(testImages, testLabels, verbose = 0)
print(acc * 100)

83.699268519


In [16]:
predictions = model.predict(testImages)

In [17]:
from sklearn.metrics import confusion_matrix
image_pred = (predictions > 0.8)
confusion_matrix(testLabels.argmax(axis=1), image_pred.argmax(axis=1))

array([[164,  48,  38],
       [ 58, 309,  83],
       [ 11,   5, 286]])

In [18]:
# serialize model to JSON
model_json = model.to_json()
with open("selfie_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("selfie_model-weights.h5")
print("Saved model to disk")
model.save("selfie_model.h5")

Saved model to disk
